# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.horizons import (
    horizons_bulk_vector_query,
    horizons_bulk_astrometry_query,
)

from jorbit import Particle

t0 = Time("2024-12-01 00:00")
t1 = Time("2025-12-01 00:00")

In [2]:
from jorbit.utils.horizons import get_observer_positions

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons
import numpy as np

from jorbit.utils.horizons import get_observer_positions
from jorbit.astrometry.sky_projection import sky_sep, on_sky
from jorbit.utils.states import SystemState
from jorbit.accelerations import create_newtonian_ephemeris_acceleration_func
from jorbit.ephemeris import Ephemeris


# gather positions and observed coordinates for Kitt Peak

t0_kp = Time("2024-12-01 00:00")
t1_kp = Time("2025-12-01 00:00")

obj = Horizons(
    id="274301",
    location="500@0",
    epochs={"start": t0_kp.tdb.iso, "stop": t1_kp.tdb.iso, "step": "2d"},
)
vecs_kp = obj.vectors(refplane="earth")
xs_kp = jnp.array([vecs_kp["x"], vecs_kp["y"], vecs_kp["z"]]).T
vs_kp = jnp.array([vecs_kp["vx"], vecs_kp["vy"], vecs_kp["vz"]]).T
times_kp = jnp.array(vecs_kp["datetime_jd"])

obj = Horizons(
    id="274301",
    location="695@399",
    epochs={"start": t0_kp.utc.iso, "stop": t1_kp.utc.iso, "step": "2d"},
)
coords_kp = obj.ephemerides(extra_precision=True, quantities="1")
ra_kp = jnp.array(coords_kp["RA"])
dec_kp = jnp.array(coords_kp["DEC"])


# gather positions and observed coordinates for Palomar

t0_pal = Time("2024-12-02 00:00")
t1_pal = Time("2025-12-02 00:00")

obj = Horizons(
    id="274301",
    location="500@0",
    epochs={"start": t0_pal.tdb.iso, "stop": t1_pal.tdb.iso, "step": "2d"},
)
vecs_pal = obj.vectors(refplane="earth")
xs_pal = jnp.array([vecs_pal["x"], vecs_pal["y"], vecs_pal["z"]]).T
vs_pal = jnp.array([vecs_pal["vx"], vecs_pal["vy"], vecs_pal["vz"]]).T
times_pal = jnp.array(vecs_pal["datetime_jd"])

obj = Horizons(
    id="274301",
    location="675@399",
    epochs={"start": t0_pal.utc.iso, "stop": t1_pal.utc.iso, "step": "2d"},
)
coords_pal = obj.ephemerides(extra_precision=True, quantities="1")
ra_pal = jnp.array(coords_pal["RA"])
dec_pal = jnp.array(coords_pal["DEC"])

# combine everything
xs = jnp.concatenate([xs_kp, xs_pal])
vs = jnp.concatenate([vs_kp, vs_pal])
times = jnp.concatenate([times_kp, times_pal])
ra = jnp.concatenate([ra_kp, ra_pal])
dec = jnp.concatenate([dec_kp, dec_pal])
observatories = ["kitt peak"] * len(xs_kp) + ["palomar"] * len(xs_pal)

order = jnp.argsort(times)
xs = xs[order]
vs = vs[order]
times = Time(times[order], format="jd", scale="tdb")
ra = ra[order]
dec = dec[order]
observatories = np.array(observatories)[order]


# get the observer positions
obs_pos = get_observer_positions(times, observatories)

eph = Ephemeris("default planets")
acc_func = create_newtonian_ephemeris_acceleration_func(eph.processor)

calc_ras, calc_decs = jax.vmap(on_sky, in_axes=(0, 0, 0, 0, None))(
    xs, vs, times.tdb.jd, obs_pos, acc_func
)

calc_skycoords = SkyCoord(ra=calc_ras, dec=calc_decs, unit=(u.rad, u.rad))
true_skycoords = SkyCoord(ra=ra * u.deg, dec=dec * u.deg)


calc_skycoords.separation(true_skycoords).to(u.arcsec)

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 231, 233, 235, 237, 239, 241, 243, 245, 247, 249, 251, 253, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 275, 277, 279, 281, 283, 285, 287, 289, 291, 293, 295, 297, 299, 301, 303, 305, 307, 309, 311, 313, 315, 317, 319, 321, 323, 325, 327, 329, 331, 333, 335, 337, 339, 341, 343, 345, 347, 349, 351, 353, 355, 357, 359, 361, 363, 365]
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70,

array([1.76283805e-06, 1.23766711e-06, 2.74384288e-06, 2.35761421e-06,
       2.20126922e-06])

In [15]:
pal_pos = get_observer_positions(Time(times_pal, format="jd", scale="tdb"), ["palomar"]*len(times_pal))
ra_p, dec_p = jax.vmap(on_sky, in_axes=(0, 0, 0, 0, None))(xs_pal, vs_pal, times_pal, pal_pos, acc_func)
s_pal = SkyCoord(ra=ra_p, dec=dec_p, unit=(u.rad, u.rad))
c_pal = SkyCoord(ra=ra_pal, dec=dec_pal, unit=(u.deg, u.deg))

s_pal.separation(c_pal).to(u.arcsec).value[:5]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182]


array([437.53047119, 447.89668909, 458.37816681, 468.96825916,
       479.65949916])

In [18]:
Time(times_pal[0], format="jd", scale="tdb").iso

'2024-12-02 00:01:09.183'

In [16]:
ra_pal[0], ra_kp[0]

(Array(152.14794954, dtype=float64), Array(152.01007338, dtype=float64))

In [12]:
get_observer_positions(times[1], observatories[1])

Array([[0.33206907, 0.8453052 , 0.36666236]], dtype=float64)

In [13]:
obs_pos[1]

Array([0.33206907, 0.8453052 , 0.36666236], dtype=float64)

In [ ]:
get_observer_positions(times[0], observatories[0])

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.horizons import (
    horizons_bulk_vector_query,
    horizons_bulk_astrometry_query,
)

from jorbit import Particle



t0_pal = Time("2024-12-02 00:00")
t1_pal = Time("2025-12-02 00:00")

obj = Horizons(
    id="274301",
    location="500@0",
    epochs={"start": t0_pal.tdb.iso, "stop": t1_pal.tdb.iso, "step": "2d"},
)
vecs_pal = obj.vectors(refplane="earth")
xs_pal = jnp.array([vecs_pal["x"], vecs_kp["y"], vecs_pal["z"]]).T
vs_pal = jnp.array([vecs_pal["vx"], vecs_pal["vy"], vecs_pal["vz"]]).T
times_pal = jnp.array(vecs_pal["datetime_jd"])

obj = Horizons(
    id="274301",
    location="675@399",
    epochs={"start": t0_pal.utc.iso, "stop": t1_pal.utc.iso, "step": "2d"},
)
coords_pal = obj.ephemerides(extra_precision=True, quantities="1")
ra_pal = jnp.array(coords_pal["RA"])
dec_pal = jnp.array(coords_pal["DEC"])

pal_pos = get_observer_positions(Time(times_pal, format="jd", scale="tdb"), ["palomar"]*len(times_pal))
ra_p, dec_p = jax.vmap(on_sky, in_axes=(0, 0, 0, 0, None))(xs_pal, vs_pal, times_pal, pal_pos, acc_func)
s_pal = SkyCoord(ra=ra_p, dec=dec_p, unit=(u.rad, u.rad))
c_pal = SkyCoord(ra=ra_pal, dec=dec_pal, unit=(u.deg, u.deg))

s_pal.separation(c_pal).to(u.arcsec).value[:5]


NameError: name 'vecs_kp' is not defined

In [2]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons
import numpy as np

from jorbit.utils.horizons import get_observer_positions
from jorbit.astrometry.sky_projection import sky_sep, on_sky
from jorbit.utils.states import SystemState
from jorbit.accelerations import create_newtonian_ephemeris_acceleration_func
from jorbit.ephemeris import Ephemeris


def test_sky_projection():
    # gather positions and observed coordinates for Kitt Peak

    t0_kp = Time("2024-12-01 00:00")
    t1_kp = Time("2025-12-01 00:00")

    obj = Horizons(
        id="274301",
        location="500@0",
        epochs={"start": t0_kp.tdb.iso, "stop": t1_kp.tdb.iso, "step": "2d"},
    )
    vecs_kp = obj.vectors(refplane="earth")
    xs_kp = jnp.array([vecs_kp["x"], vecs_kp["y"], vecs_kp["z"]]).T
    vs_kp = jnp.array([vecs_kp["vx"], vecs_kp["vy"], vecs_kp["vz"]]).T
    times_kp = jnp.array(vecs_kp["datetime_jd"])

    obj = Horizons(
        id="274301",
        location="695@399",
        epochs={"start": t0_kp.utc.iso, "stop": t1_kp.utc.iso, "step": "2d"},
    )
    coords_kp = obj.ephemerides(extra_precision=True, quantities="1")
    ra_kp = jnp.array(coords_kp["RA"])
    dec_kp = jnp.array(coords_kp["DEC"])

    # gather positions and observed coordinates for Palomar

    t0_pal = Time("2024-12-02 00:00")
    t1_pal = Time("2025-12-02 00:00")

    obj = Horizons(
        id="274301",
        location="500@0",
        epochs={"start": t0_pal.tdb.iso, "stop": t1_pal.tdb.iso, "step": "2d"},
    )
    vecs_pal = obj.vectors(refplane="earth")
    xs_pal = jnp.array([vecs_pal["x"], vecs_pal["y"], vecs_pal["z"]]).T
    vs_pal = jnp.array([vecs_pal["vx"], vecs_pal["vy"], vecs_pal["vz"]]).T
    times_pal = jnp.array(vecs_pal["datetime_jd"])

    obj = Horizons(
        id="274301",
        location="675@399",
        epochs={"start": t0_pal.utc.iso, "stop": t1_pal.utc.iso, "step": "2d"},
    )
    coords_pal = obj.ephemerides(extra_precision=True, quantities="1")
    ra_pal = jnp.array(coords_pal["RA"])
    dec_pal = jnp.array(coords_pal["DEC"])

    # combine everything
    xs = jnp.concatenate([xs_kp, xs_pal])
    vs = jnp.concatenate([vs_kp, vs_pal])
    times = jnp.concatenate([times_kp, times_pal])
    ra = jnp.concatenate([ra_kp, ra_pal])
    dec = jnp.concatenate([dec_kp, dec_pal])
    observatories = ["kitt peak"] * len(xs_kp) + ["palomar"] * len(xs_pal)

    order = jnp.argsort(times)
    xs = xs[order]
    vs = vs[order]
    times = Time(times[order], format="jd", scale="tdb")
    ra = ra[order]
    dec = dec[order]
    observatories = np.array(observatories)[order]

    # get the observer positions
    obs_pos = get_observer_positions(times, observatories)

    eph = Ephemeris("default planets")
    acc_func = create_newtonian_ephemeris_acceleration_func(eph.processor)

    calc_ras, calc_decs = jax.vmap(on_sky, in_axes=(0, 0, 0, 0, None))(
        xs, vs, times.tdb.jd, obs_pos, acc_func
    )

    calc_skycoords = SkyCoord(ra=calc_ras, dec=calc_decs, unit=(u.rad, u.rad))
    true_skycoords = SkyCoord(ra=ra * u.deg, dec=dec * u.deg)

    seps_astropy = calc_skycoords.separation(true_skycoords).to(u.arcsec)
    seps_jorbit = jax.vmap(sky_sep, in_axes=(0, 0, 0, 0))(
        calc_ras, calc_decs, ra * u.deg.to(u.rad), dec * u.deg.to(u.rad)
    )

    # test that sky_sep is doing its job
    # np.testing.assert_allclose(seps_astropy, seps_jorbit, rtol=1e-6)
    return seps_astropy, seps_jorbit

a, j = test_sky_projection()

In [4]:
a[:5].value, j[:5]

(array([1.76283805e-06, 1.23766711e-06, 2.74384288e-06, 2.35761421e-06,
        2.20126922e-06]),
 Array([1.76281602e-06, 1.23763684e-06, 2.74383635e-06, 2.35763650e-06,
        2.20127100e-06], dtype=float64))

In [7]:
np.testing.assert_allclose(a.value, j, atol=1e-6)

In [9]:
np.testing.assert_allclose(a.value, 0.0, atol=1e-4)

In [10]:
np.max(np.abs(a.value))

np.float64(2.6125593940408135e-05)